In [ ]:
import pandas as pd
import numpy as np

# Read the CSV data (assuming it's in a string or file)
# For this example, I'll assume we have it as a string from your input
# In practice, you might read from a file: df = pd.read_csv('filename.csv')
df = pd.read_csv('your_data.csv')  # Replace with actual data source

# 1. Basic Data Inspection
print("Initial Data Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())

# 2. Remove unnecessary column 'Unnamed: 0' if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# 3. Handle missing values
# Fill missing numerical values with median
numeric_columns = df.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    df[col] = df[col].fillna(df[col].median())

# Fill missing categorical values with mode
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# 4. Standardize data types
# Convert date columns to datetime
date_columns = ['dob', 'trans_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Convert time to proper format
df['trans_time'] = pd.to_datetime(df['trans_time'], format='%H:%M:%S', errors='coerce').dt.time

# Convert numerical columns
df['zip'] = df['zip'].astype(int)
df['amt'] = df['amt'].astype(float)
df['is_fraud'] = df['is_fraud'].astype(int)
df['unix_time'] = df['unix_time'].astype(int)
df['quarter'] = df['quarter'].astype(int)

# 5. Clean specific columns
# Remove leading/trailing whitespace from string columns
for col in categorical_columns:
    df[col] = df[col].str.strip()

# Standardize gender to M/F
df['gender'] = df['gender'].str.upper().replace({'MALE': 'M', 'FEMALE': 'F'})

# Clean merchant names by removing 'fraud_' prefix
df['merchant'] = df['merchant'].str.replace('fraud_', '')

# 6. Handle duplicates
# Remove exact duplicates
df = df.drop_duplicates()

# 7. Validate and clean geographical coordinates
# Ensure latitude is between -90 and 90
df['lat'] = df['lat'].clip(-90, 90)
df['merch_lat'] = df['merch_lat'].clip(-90, 90)

# Ensure longitude is between -180 and 180
df['long'] = df['long'].clip(-180, 180)
df['merch_long'] = df['merch_long'].clip(-180, 180)

# 8. Validate and clean monetary amounts
df['amt'] = df['amt'].clip(lower=0)  # Ensure no negative amounts

# 9. Standardize state abbreviations
df['state'] = df['state'].str.upper()

# 10. Add derived columns if needed
# Calculate age from DOB
current_date = pd.to_datetime('2025-03-15')  # Using current date from your context
df['age'] = ((current_date - df['dob']).dt.days / 365.25).astype(int)

# 11. Final data validation
# Check for any remaining null values
print("\nFinal Missing Values:")
print(df.isnull().sum())

# Display basic statistics
print("\nCleaned Data Statistics:")
print(df.describe())

# Display first few rows of cleaned data
print("\nFirst 5 rows of cleaned data:")
print(df.head())

# Optional: Save cleaned data
# df.to_csv('cleaned_data.csv', index=False)